In [2]:
import numpy
from scipy import stats
from matplotlib import pyplot as plot
from statsmodels.sandbox.stats.multicomp import multipletests
%matplotlib inline

ImportError: No module named 'statsmodels'

In [ ]:
with open("3.txt") as file:
    X = numpy.array([float(line.split(' ')[0]) for line in file])

with open("3.txt") as file:
    Y = numpy.array([float(line.split(' ')[1]) for line in file])

In [ ]:
print(len(X), len(Y))

In [ ]:
plot.scatter(X, Y)

По картинке видно, что выборки зависимы, можно это проверить даже с попомщью кожффициентов корреляции. Но воспользуемся критерием хи-квадрат.

In [ ]:
class Chi2Exception:
    pass

def independence_p_value(X, Y, n, m):
    epsilon = 0.01
    B = numpy.linspace(min(X) - epsilon, max(X) + epsilon, n + 1)
    C = numpy.linspace(min(Y) - epsilon, max(Y) + epsilon, m + 1)
    nu = numpy.array([[
            len([1 for x, y in zip(X, Y) if ((B[i] <= x < B[i + 1]) and (C[j] <= y < C[j + 1]))])
            for j in range(m)
        ]
        for i in range(n)
    ])

    N = numpy.sum(nu)
    ns = numpy.array([sum(nu[i,:]) for i in range(n)])
    ms = numpy.array([sum(nu[:,j]) for j in range(m)])
    for x in ns:
        for y in ms:
            if x * y / N < 5:
                raise Chi2Exception
    statistic = N * numpy.sum([[
            (nu[i][j] - (ns[i] * ms[j]) / N) ** 2 / (ns[i] * ms[j])
            for j in range(m)
        ]
        for i in range(n)
    ])
    return stats.chi2.sf(statistic, df=(n-1)*(m-1))

In [ ]:
alpha=0.1

try:
    print(independence_p_value(X, Y, 3, 3))
except:
    print("=(")

p-value хи-квадрат очень маленькое. Выбрав $alpha=0.1$ (как в условии), гипотезу о независимости можно отвергнуть.

In [ ]:
methods = [
    (lambda sample: stats.shapiro(sample)[1]),
    (lambda sample: 
        stats.kstest(
            sample, lambda x: stats.norm.cdf(
                x, loc=stats.norm.fit(sample)[0],
                scale=stats.norm.fit(sample)[1]
            )
        )[1]
    ),
    (lambda sample: stats.jarque_bera(sample)[1]),
    (lambda sample: stats.skewtest(sample)[1])
]

In [ ]:
X_p_values = [method(X) for method in methods]
Y_p_values = [method(Y) for method in methods]
X_answers = multipletests(X_p_values, method="holm", alpha=alpha)[0]
Y_answers = multipletests(Y_p_values, method="holm", alpha=alpha)[0]